In [1]:
import math
import csv
import folium

from geopy.distance import geodesic
from geopy.geocoders import Nominatim

In [2]:
def get_hexagon_row_coord(start_coord, radius, count, coord_data):
    if count== 0:
        return coord_data
    else:
        next_coord = geodesic(meters=radius*2).destination(start_coord, 90)
        row = {
            'coordinate': (next_coord.latitude, next_coord.longitude),
        }
        coord_data.append(row)
        return get_hexagon_row_coord(next_coord, radius, count-1, coord_data)

In [3]:
def row_mapper(start_coord, radius, row_count, height_count, coord_data):
    height = radius * math.sqrt(3)
    if height_count == 0:
        return coord_data
    elif height_count % 2 != 0:
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), 90
        )
        coord_data = get_hexagon_row_coord(next_coord, radius, row_count, coord_data)
    else:
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), 270
        )
        coord_data = get_hexagon_row_coord(next_coord, radius, row_count, coord_data)
    return row_mapper(next_coord, radius, row_count, height_count-1, coord_data)

In [4]:
def data_to_csv(data):
    file_path = 'coordinates-list/hexagonal-packing-v1.csv'
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        for row in data:
            writer.writerow(row)
        print(f"Coordinates saved to {file_path}")

In [10]:
r = 500
start_point = (-8.429957624116502, 115.03252775649212)
row_count = 50
height_count = 17
coordinates = row_mapper(start_point, r, row_count, height_count, [])

In [11]:
print(coordinates)

[{'coordinate': (-8.43778787037442, 115.04614895353889)}, {'coordinate': (-8.437787765236736, 115.055229751567)}, {'coordinate': (-8.437787660099051, 115.06431054959266)}, {'coordinate': (-8.437787554961368, 115.07339134761587)}, {'coordinate': (-8.437787449823686, 115.08247214563661)}, {'coordinate': (-8.437787344686006, 115.0915529436549)}, {'coordinate': (-8.43778723954833, 115.10063374167073)}, {'coordinate': (-8.437787134410652, 115.10971453968412)}, {'coordinate': (-8.437787029272975, 115.11879533769505)}, {'coordinate': (-8.437786924135303, 115.12787613570352)}, {'coordinate': (-8.43778681899763, 115.13695693370953)}, {'coordinate': (-8.43778671385996, 115.14603773171308)}, {'coordinate': (-8.437786608722291, 115.15511852971419)}, {'coordinate': (-8.437786503584622, 115.16419932771284)}, {'coordinate': (-8.437786398446955, 115.17328012570903)}, {'coordinate': (-8.43778629330929, 115.18236092370276)}, {'coordinate': (-8.437786188171627, 115.19144172169403)}, {'coordinate': (-8.43

In [12]:
data_to_csv(coordinates)

Coordinates saved to coordinates-list/hexagonal-packing-v1.csv


In [13]:
world_map = folium.Map(location=start_point, zoom_start=15)

for coord in coordinates:
    folium.Marker(
        location=coord['coordinate'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(world_map)

In [14]:
world_map.save('test-map.html')